# 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import torch
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/data/"

SEED = 42 # 시드값

# 데이터 블러오기
train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터

# 결측치 처리
age_mean = train["age"].mean()
fare_median = train["fare"].median()
cabin_unk = "UNK"
embarked_mode = train["embarked"].mode()[0]
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

# 특성으로 사용할 변수 선택
cols = ["age","sibsp","parch","fare","pclass","gender","embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy()

# 범주형 변수 원핫인코딩
cols = ['gender','embarked']
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft = scaler.transform(train_ft)
test_ft = scaler.transform(test_ft)

# 정답 데이터
target = train["survived"].to_numpy().reshape(-1,1) # 정답데이터 2차원 변경

train_ft.shape, test_ft.shape, target.shape

Mounted at /content/drive


((916, 10), (393, 10), (916, 1))

# 재현성 함수(Reproduction)

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# 데이터셋 클래스 구현
- 미니 배치 학습을 위한 데이터셋 클래스 구현 > train, target
- dataloader클래스를 사용해서 미니 배치 학습이 가능한 상태 만들기

In [ ]:
class TitanicDataset(torch.utils.data.Dataset): # 데이터셋 클래스 구현
  def __init__(self, x, y = None):
    self.x = x
    self.y = y

  def __len__(self): # 샘플 수를 반환해주는 역할
    return len(self.x)

  def __getitem__(self, idx): # 인덱싱+슬라이싱가능한 형태
    item = {}
    item['x'] = torch.Tensor(self.x[idx])
    if self.y is not None: # y값이 있을 경우
      item['y'] = torch.Tensor(self.y[idx])
    return item

- 결과 확인하기

In [ ]:
train_ft.shape[0]

916

In [ ]:
dt = TitanicDataset(train_ft, target) # 학습데이터, 정답데이터
dt[0] # 범위: -916~915

{'x': tensor([0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000]),
 'y': tensor([0.])}

In [ ]:
dl = torch.utils.data.DataLoader(dt, batch_size = 5, shuffle = False) # batch_size 개수 만큼 저장
batch = next(iter(dl)) # dl은 반복 가능한 객체가 아니므로 iter로 변환

In [ ]:
batch

{'x': tensor([[0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
          0.0000],
         [0.4238, 0.0000, 0.0000, 0.0157, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
          1.0000],
         [0.3611, 0.3750, 0.1111, 0.0430, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
          1.0000],
         [0.2233, 0.1250, 0.1111, 0.0254, 0.5000, 1.0000, 0.0000, 0.0000, 0.0000,
          1.0000],
         [0.5991, 0.0000, 0.0000, 0.0518, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
          1.0000]]),
 'y': tensor([[0.],
         [0.],
         [0.],
         [1.],
         [1.]])}

In [ ]:
batch['x']

tensor([[0.8873, 0.0000, 0.0000, 0.0966, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000,
         0.0000],
        [0.4238, 0.0000, 0.0000, 0.0157, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000],
        [0.3611, 0.3750, 0.1111, 0.0430, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000],
        [0.2233, 0.1250, 0.1111, 0.0254, 0.5000, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000],
        [0.5991, 0.0000, 0.0000, 0.0518, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         1.0000]])

In [ ]:
len(dl) # train_ft의 개수에서 batch_size 나눈 개수

184

# 인공 신경망 모델 클래스 구현
- BN : 배치정규화
- FC(Linear) > BN > H(활성화)

In [ ]:
train_ft.shape[1]

10

In [ ]:
class Net(torch.nn.Module):
  def __init__(self, n_features):
    super().__init__() # 부모의 __init__메서드 받아야함
    self.seq = torch.nn.Sequential(
        torch.nn.Linear(n_features, 12), # FC: 선형변환
        torch.nn.BatchNorm1d(12), # BN: 배치 정규화
        torch.nn.LeakyReLU(), # H: 활성화
        torch.nn.Linear(12, 8),
        torch.nn.BatchNorm1d(8),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(8, 4),
        torch.nn.BatchNorm1d(4),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(4, 1)
    )
  def forward(self, x): # 생성된 신경망 객체 사용
    return self.seq(x)

- 결과 확인하기

In [ ]:
model = Net(train_ft.shape[1])
model(batch['x'])

tensor([[-0.6196],
        [ 0.8342]], grad_fn=<AddmmBackward0>)

# 하이퍼파라미터 정의
- 손실함수, 배치사이즈, 디바이스, 피처 개수

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters()) # 최적화: Adam
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
n_features = train_ft.shape[1]

# 학습데이터에 대한 배치단위 loop 구현
- epoch

In [ ]:
model = Net(n_features).to(device) # 인공신경망 입력데이터 피처 개수
train_dt = TitanicDataset(train_ft, target) # dt 구현
train_dl = torch.utils.data.DataLoader(train_dt, batch_size = batch_size, shuffle = True) # dl 구현, 학습 데이터 > 셔플 o

In [ ]:
reset_seeds(SEED)

for _ in range(10):
  epoch_loss = 0
  for batch in train_dl:
    pred = model(batch['x']).to(device) # 예측값
    loss = loss_fn(pred, batch['y'].to(device)) # 손실함수

    optimizer.zero_grad() # 누적되는 기울기 > 기울기 0초기화
    loss.backward() # 역전파 실행
    optimizer.step() # 가중치 업데이트
    epoch_loss += loss.item() # gpu환경에서 실행

  epoch_loss /= len(train_dl) # epoch_loss비율
  print(epoch_loss)

0.6511606779591791
0.5812930978577713
0.5402021387527729
0.5049006178461272
0.483206484852166
0.4613448032017412
0.45694437109190844
0.43889733737912673
0.43048577164781504
0.41539643345208005


# 테스트 데이터에 대한 배치단위 예측 loop구현

In [ ]:
test_dt = TitanicDataset(test_ft)
test_dl = torch.utils.data.DataLoader(test_dt, batch_size = batch_size, shuffle = False) # 테스트 데이터 > 셔플 x

In [ ]:
model.eval() # 평가모드

pred_list = []
act = torch.nn.Sigmoid() # H: 활성화
with torch.no_grad(): # 경사추적할 필요 없기때문에
  for batch in test_dl:
    pred = model(batch['x'].to(device))
    pred = act(pred)
    pred = pred.to('cpu').numpy() # gpu에 있는 pred를 cpu에서 numpy로 저장
    pred_list.append(pred)

- 데이터 합치기

In [ ]:
pred = np.concatenate(pred_list)
pred.shape

(393, 1)

# workflow정리
- 데이터셋 구현 > 결과 확인하기(dt, dl, batch)
- 인공신경망모델 구현 > 결과 확인하기(model)
- 하이퍼파라미터 정의 > 손실함수, 배치사이즈, 디바이스, 피처 개수 etc...
- 학습데이터에 대한 배치단위 loop 구현 > model.train(), epoch_loss, optimizer, backwar, step
- 테스트 데이터에 대한 배치단위 예측 loop구현 > 예측값 합치기 (model.eval(), act, pred)